## 1. 모듈삽입

In [166]:
import os
import re
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import keras.backend.tensorflow_backend as K
from tensorflow.python.framework import ops as tf_ops
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import warnings 
from PIL import Image
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

## 2. 데이터 설명 파일 불러오기

In [85]:
# 데이터 프레임 불러오기

total_data_final_2 = pd.read_csv('./total_data_final_2.csv')

In [86]:
# 필요한 부분의 data frame 설정

total_data_final_2.columns
df = total_data_final_2[['folder_name','image_num','category_label','category_name',
                         'category_type','texture','shape','style']]
df

,folder_name,image_num,category_label,category_name,category_type,texture,shape,style
0,Sheer_Pleated-Front_Blouse,img_00000001.jpg,3,Blouse,1,NaN,shift,NaN
1,Sheer_Pleated-Front_Blouse,img_00000002.jpg,3,Blouse,1,NaN,shift,NaN
2,Sheer_Pleated-Front_Blouse,img_00000003.jpg,3,Blouse,1,NaN,slip,NaN
3,Sheer_Pleated-Front_Blouse,img_00000004.jpg,3,Blouse,1,NaN,NaN,NaN
4,Sheer_Pleated-Front_Blouse,img_00000005.jpg,3,Blouse,1,NaN,NaN,shark
...,...,...,...,...,...,...,...,...
289103,Paisley_Print_Babydoll_Dress,img_00000050.jpg,41,Dress,3,palm springs,print shift,NaN
289104,Paisley_Print_Babydoll_Dress,img_00000051.jpg,41,Dress,3,palm,NaN,NaN
289105,Paisley_Print_Babydoll_Dress,img_00000052.jpg,41,Dress,3,palm,NaN,NaN
289106,Paisley_Print_Babydoll_Dress,img_00000053.jpg,41,Dress,3,palm springs,print shift,NaN


## 3. 이미지 데이터 불러오기

In [ ]:
# 이미지를 불러오는 방법 2가지

# 1. plt 사용
# plt.imread('파일이름.jpg') : 숫자 array로 불러옴
# plt.imshow : 위의 숫자를 이미지로 보여줌

# 2. pillow 사용
# Image.open('파일이름.jpg') : 이미지로 불러옴
# np.array(위의 이미지) : 위의 이미지를 숫자로 보여줌

In [65]:
# 우선 예시로 이미지 열어봄

mm = Image.open('D:/Project/img/Sheer_Pleated-Front_Blouse/img_00000002.jpg')
print(mm.size) 
print(mm.format)
# np.array(mm)

# 이미지 사이즈 다 달라서 학습시 동일하게 맞춰주는 작업 필요!!

(205, 300)
JPEG


## 4. texture 분류모델

In [32]:
# texture, style, shape 각각 따로 NA 처리하고 분류 모델 만들기

### 4.1. NA, category_typ=3 처리 

In [118]:
# texture NA, category_type=3 삭제 : 64000 row 남음


# 불필요 col 삭제
df_t = df.drop(['style','shape'], axis=1)

# texture NA 삭제
df_t = df_t.dropna(axis=0, how='any')
df_t.shape #100000

# category_type = 3 삭제
df_t = df_t[df_t.category_type != 3]
df_t.shape

# dropna 하면서 drop된 행의 index 비어버림 : index 초기화
df_t = df_t.reset_index(drop=True)
df_t

,folder_name,image_num,category_label,category_name,category_type,texture
0,Sheer_Pleated-Front_Blouse,img_00000009.jpg,3,Blouse,1,southwestern-patterned
1,Sheer_Pleated-Front_Blouse,img_00000035.jpg,3,Blouse,1,polka dot
2,Sheer_Pleated-Front_Blouse,img_00000036.jpg,3,Blouse,1,tribal
3,Sheer_Pleated-Front_Blouse,img_00000038.jpg,3,Blouse,1,chiffon floral
4,Sheer_Pleated-Front_Blouse,img_00000040.jpg,3,Blouse,1,zig
...,...,...,...,...,...,...
64746,Destroyed_Denim_Cutoffs,img_00000015.jpg,24,Cutoffs,2,diamond
64747,Destroyed_Denim_Cutoffs,img_00000016.jpg,24,Cutoffs,2,diamond print
64748,Destroyed_Denim_Cutoffs,img_00000017.jpg,24,Cutoffs,2,diamond print
64749,Destroyed_Denim_Cutoffs,img_00000018.jpg,24,Cutoffs,2,paint


### 4.2. 데이터 확보

In [ ]:
# X : 이미지 픽셀 값
# y : texture 이름 one hot encofing 

In [124]:
# X에 해당되는 데이터를 모아보자 : 이미지 픽셀 데이터 
# 사용할 df: df_t


img_w = 64
img_h = 64

pixels = img_w * img_h * 3

t_X_data = []

for _ in range(df_rm_t.shape[0]):
    
    img_dir = df_t['folder_name'][_] 
    img_name = df_t['image_num'][_] 
    classes = df_t['texture'].unique()
    full_dir = str('D:/Project/img/' + img_dir + '/' + img_name) #이미지 주소
    #print(full_dir)
    
    img = Image.open(full_dir)
    img = img.convert("RGB")
    img = img.resize((img_w, img_h))
    data = np.array(img)
    
    t_X_data.append(data)
    
    if _ % 1000 == 0:
        print(_)

    
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000


In [125]:
# y에 해당되는 데이터를 모아보자 : texture 에 대한 값, one-hot encoding 필요
# 사용할 df = df_t


img_w = 64
img_h = 64

pixels = img_w * img_h * 3

# y 데이터
t_y_data = df_t['texture']
t_y_data = pd.get_dummies(t_y_data, columns=['texture'])
t_y_data



,abstract chevron,abstract diamond,abstract floral print,abstract geo,abstract paisley,abstract print,abstract printed,abstract stripe,animal,animal print,...,stripes,structured,tonal,tribal,tribal-inspired,two-tone,varsity-striped,watercolor,zig,zigzag
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64746,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64747,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64748,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64749,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 4.3. 분류모델 

In [148]:
# 1. 데이터
X = np.array(t_X_data)
y = np.array(t_y_data)

X_train, X_test, y_train, y_test = train_test_split(X,y)
xy = (X_train, X_test, y_train, y_test)
np.save("texture_data.npy", xy)  # 데이터 저장까진 되는데 load 오류남

64751


In [149]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [151]:
print(X_train.shape)
print(X_train.shape[0])

(48563, 64, 64, 3)
48563


In [153]:
categories = t_y_data.columns
nb_classes = len(categories)

In [154]:
#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

In [167]:
with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_dir = './model'
    
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    
    model_path = model_dir + '/multi_img_classification.model'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=6)

AttributeError: module 'keras.backend.tensorflow_backend' has no attribute 'tf_ops'

## 4.2. shape
## 4.2.1. NA, category_type=3 처리

In [34]:
# shape NA, category_type=3 삭제 : 93000 row 남음


# 불필요 col 삭제
df_rm_sh = df.drop(['style','texture'], axis=1)

# texture NA 삭제
df_rm_sh = df_rm_sh.dropna(axis=0, how='any')
df_rm_sh.shape #140000

# category_type = 3 삭제
df_rm_sh = df_rm_sh[df_rm_sh.category_type != 3]
df_rm_sh.shape

(93093, 6)

## 4.2.2. 분류 모델

## 4.3 style
## 4.3.1. NA, category_type=3 처리

In [35]:
# style NA, category_type=3 삭제 : 88000 row 남음


# 불필요 col 삭제
df_rm_st = df.drop(['shape','texture'], axis=1)

# texture NA 삭제
df_rm_st = df_rm_st.dropna(axis=0, how='any')
df_rm_st.shape #120000

# category_type = 3 삭제
df_rm_st = df_rm_st[df_rm_st.category_type != 3]
df_rm_st.shape

(88802, 6)

## 4.3.2. 분류 모델

## 4. 이미지 데이터 픽셀화 : image_name/픽셀값

In [ ]:
img_dir = df['folder_name'][인덱스]
img_name = df['image_num'][인덱스]

categories = df['해당 column'].unique()
nb_classes = len(categories)

img_w = 64
img_h = 64

pixels = img_w * img_h * 3

X=[]
Y=[]

for idx, cat in enumerate(categories):
    
    # enumerate : 인덱스 번호와 컬렉션의 원소를 tuple 형태로 반환
    
    # one-hot 돌리기
    label= [0 for i in ragne(nb_classes)]
    label[ide] = 1
    
    img_dir = cate